In [1]:
!pip install datasets transformers[sentencepiece]==4.28.0
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.4.1 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
cuml 23.4.1 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
dask-cudf 23.4.1 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is

# Libraries & Functions

In [2]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [4]:
import torch
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
from tokenizers import BertWordPieceTokenizer
from transformers import AutoTokenizer, DataCollatorWithPadding, BertTokenizerFast, DistilBertTokenizerFast,  TrainingArguments, Trainer, AutoModelForSequenceClassification
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_predict

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
!git config --global user.email "viktor.domazetoski@hotmail.com"
!git config --global user.name "ViktorDo1"

from huggingface_hub import notebook_login
notebook_login()

In [6]:
model_names = ["BERT", "BioBERT"]
checkpoint_names = ["bert-base-cased", "dmis-lab/biobert-base-cased-v1.2"]

In [7]:
def prepare_data(X, y):
    data = []
    for i, (sequence, label) in enumerate(zip(X, y)):
        data.append( 
            {"text":sequence, 
            "label":label,
            "idx":i
            })
    df = pd.DataFrame(data)
    return Dataset.from_pandas(df)

In [8]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length = 512)

In [9]:
def calculate_scores(y_test, y_pred, average = "binary"):
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average = average)
    recall = recall_score(y_test, y_pred, average = average)
    f1 = f1_score(y_test, y_pred, average = average)
    auc = roc_auc_score(pd.get_dummies(y_test), pd.get_dummies(y_pred), average = average)
    return [accuracy, precision, recall, f1, auc]

In [10]:
description_threshold = 100

# Input Data

In [11]:
raw_datasets = dict()

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

# working_dir = "drive/MyDrive/Family_Text_Data/"

# !ls "drive/MyDrive/Family_Text_Data/"

## POWO Dataset

In [13]:
working_dir = "..//input//powo-family//" 

In [14]:
df_POWO_Fam =  pd.read_excel(working_dir + "POWO_Family.xlsx")
POWO_Filter = df_POWO_Fam["family"].value_counts().keys()[df_POWO_Fam["family"].value_counts().values>=description_threshold].values
df_POWO_Fam = df_POWO_Fam[df_POWO_Fam["family"].apply(lambda x: x in POWO_Filter)]
raw_datasets["POWO"] = df_POWO_Fam

In [15]:
df_POWO_Fam

,name,authors,POWO_ids,POWO_id_N,Language,description,QA_description,BERT_description,BOW_description,work_ID,species,family
0,Aa argyrolepis,Rchb.f.,morphologyGeneralHabit,1,es,Hierba,herb,herb,herb,158783,Aa argyrolepis,Orchidaceae
1,Aa colombiana,Schltr.,morphologyGeneralHabit,1,es,Hierba,herb,herb,herb,158785,Aa colombiana,Orchidaceae
2,Aa denticulata,Schltr.,morphologyGeneralHabit,1,es,Hierba,herb,herb,herb,158786,Aa denticulata,Orchidaceae
3,Aa leucantha,(Rchb.f.) Schltr.,morphologyGeneralHabit,1,es,Hierba,herb,herb,herb,158789,Aa leucantha,Orchidaceae
4,Aa maderoi,Schltr.,morphologyGeneralHabit,1,es,Hierba,herb,herb,herb,158792,Aa maderoi,Orchidaceae
...,...,...,...,...,...,...,...,...,...,...,...,...
54500,Zygostates chaparensis,Toscano & R.Vásquez,vegetativeMultiplicationPseudobulb;morphologyR...,16,en,"Pseudobulb unifoliate, ovoid, 3 – 5 × 2.5 – 3&...","pseudobulb unifoliate , ovoid , 3 - 5 x 2.5 - ...",pseudobulb unifoliate ovoid mm lateral sepals ...,pseudobulb unifoliate ovoid mm lateral sepals ...,165424,Zygostates chaparensis,Orchidaceae
54501,Zygotritonia nyassana,Mildbr.,vegetativeMultiplicationCorm;morphologyLeaf;mo...,19,en,Corms 15–20 cm in diameter; tunics of moderate...,corms 15-20 cm in diameter ; tunics of moderat...,corms cm in diameter tunics of moderately coar...,corms cm diameter tunics moderately coarse ret...,153727,Zygotritonia nyassana,Iridaceae
54502,Zygotritonia praecox,Stapf,vegetativeMultiplicationCorm;morphologyReprodu...,4,en,"Corm about 1 in. diam., covered with reticulat...","corm about 1 in . diam. , covered with reticul...",corm about in diam covered with reticulatefibr...,corm diam covered reticulatefibrous tunics cap...,153728,Zygotritonia praecox,Iridaceae
54503,× Agropogon lutosus,(Poir.) P.Fourn.,morphologyReproductiveInflorescenceBractGlume;...,7,en,Glumes persistent; similar; exceeding apex of ...,glumes persistent ; similar ; exceeding apex o...,glumes persistent similar exceeding apex of fl...,glumes persistent similar exceeding apex flore...,47395,× Agropogon lutosus,Poaceae


## WIKI Dataset

In [16]:
working_dir = "..//input//wiki-family//" 

In [17]:
df_WIKI_Fam =  pd.read_excel(working_dir + "WIKI_Family.xlsx")
WIKI_Filter = df_WIKI_Fam["family"].value_counts().keys()[df_WIKI_Fam["family"].value_counts().values>=description_threshold].values
df_WIKI_Fam = df_WIKI_Fam[df_WIKI_Fam["family"].apply(lambda x: x in WIKI_Filter)]
raw_datasets["WIKI"] = df_WIKI_Fam

In [18]:
df_WIKI_Fam

,name,authors,WIKI_ids,WIKI_id_N,Language,description,QA_description,BERT_description,BOW_description,work_ID,species,family
0,Aa achalensis,"Schltr., 1920",Summary;Summary,2,en,Aa achalensis is a species of orchid in the ge...,aa achalensis is a species of orchid in the ge...,aa achalensis is species of orchid in the genu...,aa achalensis species orchid genus aa referenc...,102932,Aa achalensis,Orchidaceae
1,Aa argyrolepis,"Rchb.f., 1854",Summary;References;Summary;References,4,en,Aa argyrolepis is an orchid in the genus Aa. ...,aa argyrolepis is an orchid in the genus aa . ...,aa argyrolepis is an orchid in the genus aa it...,aa argyrolepis orchid genus aa grows altitudes...,158783,Aa argyrolepis,Orchidaceae
2,Aa aurantiaca,D. Trujillo (2011)[1],Summary;Summary,2,en,Aa aurantiaca is a species of orchid in the ge...,aa aurantiaca is a species of orchid in the ge...,aa aurantiaca is species of orchid in the genu...,aa aurantiaca species orchid genus aa native p...,163995,Aa aurantiaca,Orchidaceae
3,Aa calceata,"Schltr., 1912",Summary;Summary,2,en,Aa calceata is a species of orchid in the genu...,aa calceata is a species of orchid in the genu...,aa calceata is species of orchid in the genus ...,aa calceata species orchid genus aait found bo...,158784,Aa calceata,Orchidaceae
4,Aa colombiana,Schltr.,Summary;Summary,2,en,Aa colombiana is a species of orchid in the ge...,aa colombiana is a species of orchid in the ge...,aa colombiana is species of orchid in the genu...,aa colombiana species orchid genus aa found co...,158785,Aa colombiana,Orchidaceae
...,...,...,...,...,...,...,...,...,...,...,...,...
55694,Zygosepalum labiosum,(Rich.) C.Schweinf.,Summary;Description,2,en,Zygosepalum labiosum is an epiphytic orchid fo...,zygosepalum labiosum is an epiphytic orchid fo...,zygosepalum labiosum is an epiphytic orchid fo...,zygosepalum labiosum epiphytic orchid found so...,116592,Zygosepalum labiosum,Orchidaceae
55695,Zygostigma australe,(Cham. & Schltdl.) Griseb.,Summary,1,en,Zygostigma australe is a species of flowering ...,zygostigma australe is a species of flowering ...,zygostigma australe is species of flowering pl...,zygostigma australe species flowering plant fa...,102870,Zygostigma australe,Gentianaceae
55696,Zygotritonia bongensis,(Pax) Mildbr.,Summary;Morphology;Distribution,3,en,Zygotritonia bongensis is a perennial herb of ...,zygotritonia bongensis is a perennial herb of ...,zygotritonia bongensis is perennial herb of th...,zygotritonia bongensis perennial herb iridacea...,78048,Zygotritonia bongensis,Iridaceae
55697,Zyzyxia lundellii,"(H.Robinson) Strother, 1991",Summary;Description and distribution;Naming,3,en,Zyzyxia is a genus of tropical shrubs in the f...,zyzyxia is a genus of tropical shrubs in the f...,zyzyxia is genus of tropical shrubs in the fam...,zyzyxia genus tropical shrubs family asteracea...,352453,Zyzyxia lundellii,Compositae


## Preprocess Data

In [19]:
print("Number of families per dataset:")
for dataset_name in list(raw_datasets.keys()):
    print("\t", dataset_name, raw_datasets[dataset_name]["family"].nunique())

Number of families per dataset:
	 POWO 68
	 WIKI 98


In [20]:
preprocessed_datasets_dict = {}

In [21]:
for dataset_name in list(raw_datasets.keys()):
    labelencoder = LabelEncoder()
    raw_datasets[dataset_name]["family_encoded"] = labelencoder.fit_transform(raw_datasets[dataset_name]["family"])

    X_train, X_test, \
    y_train, y_test, \
    indices_train, indices_test \
    = train_test_split(raw_datasets[dataset_name]["BERT_description"], raw_datasets[dataset_name]["family_encoded"], np.arange(len(raw_datasets[dataset_name])), test_size=0.25, random_state=42)

    preprocessed_datasets_dict[dataset_name] = DatasetDict()
    preprocessed_datasets_dict[dataset_name]["train"] = prepare_data(X_train, y_train)
    preprocessed_datasets_dict[dataset_name]["validation"] = prepare_data(X_test, y_test)

In [22]:
for dataset_name in list(raw_datasets.keys()):
    print(dataset_name)
    print(preprocessed_datasets_dict[dataset_name])
    print()

POWO
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 37330
    })
    validation: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 12444
    })
})

WIKI
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 37329
    })
    validation: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 12444
    })
})



In [23]:
tokenized_datasets_dict = {}

In [24]:
tokenizer_dict = {}

for model_name, model_checkpoint in zip(model_names, checkpoint_names):
    for dataset_name in list(raw_datasets.keys()):
        tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
        tokenizer_dict[model_name] = AutoTokenizer.from_pretrained(model_checkpoint)

        tokenized_datasets_dict[dataset_name, model_name] = preprocessed_datasets_dict[dataset_name].map(tokenize_function, batched=True)
        tokenized_datasets_dict[dataset_name, model_name] = tokenized_datasets_dict[dataset_name, model_name].remove_columns(["text", "idx"])
        tokenized_datasets_dict[dataset_name, model_name] = tokenized_datasets_dict[dataset_name, model_name].rename_column("label", "labels")
        tokenized_datasets_dict[dataset_name, model_name].set_format("torch")

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

In [25]:
data_collator_dict = {}
for model_name, model_checkpoint in zip(model_names, checkpoint_names):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    data_collator_dict[model_name] = DataCollatorWithPadding(tokenizer=tokenizer)

# Classical ML Models

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

results_list = []

for dataset_name in list(raw_datasets.keys()):
    print(dataset_name)
    X_train, X_test, \
    y_train, y_test, \
    indices_train, indices_test \
          = train_test_split(raw_datasets[dataset_name]["BOW_description"], raw_datasets[dataset_name]["family"].values, np.arange(len(raw_datasets[dataset_name])), test_size=0.25, random_state=42)

    vectorizer = CountVectorizer(max_features = 1000)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    lr_mod = LogisticRegression(max_iter=1000, penalty='l2', class_weight='balanced').fit(X_train, y_train)
    y_predict = lr_mod.predict(X_test)

    results = calculate_scores(y_test, y_predict, average = "macro")

    results_list.append([dataset_name] + results + ["Logistic Regression"])
      
df_results_lr = pd.DataFrame(results_list, columns=["Dataset", "Accuracy", "Precision", "Recall", "F1-Score", "AUC", "Model"])

POWO
WIKI


In [27]:
df_results_lr

,Dataset,Accuracy,Precision,Recall,F1-Score,AUC,Model
0,POWO,0.719222,0.567171,0.482209,0.473579,0.739020,Logistic Regression
1,WIKI,0.692864,0.497855,0.530788,0.505541,0.763814,Logistic Regression


# Large Language Models

## Training

In [28]:
model_dict = {}

In [29]:
if torch.cuda.is_available():  # Tell PyTorch to use the GPU. 
    device = torch.device("cuda") 
    print('There are %d GPU(s) available.' % torch.cuda.device_count()) 
    print('We will use the GPU:', torch.cuda.get_device_name(0)) # If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [30]:
for model_name, model_checkpoint in zip(model_names[:1], checkpoint_names[:1]):
    for dataset_name in list(raw_datasets.keys())[:1]:
        print(model_name, dataset_name)

        model_dict[dataset_name, model_name] = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=len(np.unique(tokenized_datasets_dict[dataset_name, model_name]["validation"]["labels"])))

        training_args = TrainingArguments(
            output_dir = "{}-finetuned-family-{}".format(model_name, dataset_name),
            learning_rate = 2e-5,
            per_device_train_batch_size = 16,
            per_device_eval_batch_size = 16,
            num_train_epochs = 3,
            weight_decay = 0.01,
            evaluation_strategy = "epoch",
            # push_to_hub=True,
            # fp16 = True
        )

        trainer = Trainer(
            model = model_dict[dataset_name, model_name],
            args = training_args,
            train_dataset = tokenized_datasets_dict[dataset_name, model_name]["train"],
            eval_dataset = tokenized_datasets_dict[dataset_name, model_name]["validation"],
            tokenizer = tokenizer_dict[model_name],
            data_collator = data_collator_dict[model_name]
        )

        trainer.train()

        # trainer.push_to_hub()

BERT POWO


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.211100,1.110468
2,0.988900,0.974370
3,0.879500,0.931477


In [32]:
results_list = []

for model_name, model_checkpoint in zip(model_names[:1], checkpoint_names[:1]):
    for dataset_name in list(raw_datasets.keys())[:1]:
        eval_dataloader = DataLoader(
              tokenized_datasets_dict[dataset_name, model_name]["validation"], batch_size=8, collate_fn=data_collator_dict[model_name]
        )

        logit_list = []
        prediction_list = []
        labels_list = []
        model_dict[dataset_name, model_name].eval()
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model_dict[dataset_name, model_name](**batch)

            labels = batch["labels"]
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            logit_list.append(logits.cpu().detach().numpy())
            prediction_list.append(predictions.cpu().detach().numpy())
            labels_list.append(labels.cpu().detach().numpy())

        y_true = np.array([item for sublist in labels_list for item in sublist])
        y_predict = np.array([item for sublist in prediction_list for item in sublist])

        results = calculate_scores(y_true, y_predict, average = "macro")

        results_list.append([dataset_name] + results + [model_name])

df_results = pd.DataFrame(results_list, columns=["Dataset", "Accuracy", "Precision", "Recall", "F1-Score", "AUC", "Model"])

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
df_results

,Dataset,Accuracy,Precision,Recall,F1-Score,AUC,Model
0,POWO,0.764465,0.542451,0.408002,0.421482,0.526382,BERT
